## Demo of implementing RAG
This Jupyter notebook is for an initial implementation of RAG, following the tutorial from [Hugging Face](https://huggingface.co/blog/ngxson/make-your-own-rag).

First we begin by loading our dataset

In [1]:
dataset = []
with open('cat-facts.txt', 'r') as file:
  dataset = file.readlines()
  print(f'Loaded {len(dataset)} entries')

dataset[0:5]  # Display the first 5 entries

Loaded 150 entries


['On average, cats spend 2/3 of every day sleeping. That means a nine-year-old cat has been awake for only three years of its life.\n',
 'Unlike dogs, cats do not have a sweet tooth. Scientists believe this is due to a mutation in a key taste receptor.\n',
 'When a cat chases its prey, it keeps its head level. Dogs and humans bob their heads up and down.\n',
 'The technical term for a cat’s hairball is a “bezoar.”\n',
 'A group of cats is called a “clowder.”\n']

## Implementing vector database

We have to convert our plain text to vectors, to be able to use vector similarity search, rather than keyword search (not a viable alternative)
<br><br/>
To do this, we need to have selected the models we are going to work with. For convenience, we will use Ollama models, as they are free and can be run locally with minimal setup.

In [2]:
import ollama

EMBEDDING_MODEL = 'hf.co/CompendiumLabs/bge-base-en-v1.5-gguf'
LANGUAGE_MODEL = 'hf.co/bartowski/Llama-3.2-1B-Instruct-GGUF'

In [3]:
# Each element in the VECTOR_DB will be a tuple (chunk, embedding)
VECTOR_DB = []

def add_chunk_to_database(chunk):
  embedding = ollama.embed(model=EMBEDDING_MODEL, input=chunk)['embeddings'][0]
  VECTOR_DB.append((chunk, embedding))

We will assume each line in our database to be one chunk. Let us now calculate the embeddings and also add them to the VECTOR_DB list

In [5]:
# Let us check if the vectorization has already been done
try:
  with open('vector_db.txt', 'r') as file:
    for line in file:
      chunk, embedding_str = line.strip().split('\t')
      embedding = list(map(float, embedding_str.split(',')))
      VECTOR_DB.append((chunk, embedding))
  print(f'Loaded {len(VECTOR_DB)} entries from vector_db.txt')
  vectorized_dataset_loaded = True
except FileNotFoundError:
  print('vector_db.txt not found, proceeding to vectorize the dataset')
  for i, chunk in enumerate(dataset):
    add_chunk_to_database(chunk)
    print(f'Added chunk {i+1}/{len(dataset)} to the database')
  vectorized_dataset_loaded = False
    
  print(f'Added {len(VECTOR_DB)} chunks to the database')

Loaded 300 entries from vector_db.txt


Let us save our vectorized database

In [6]:
print(f"The vectors in this list of tuples are {(type(VECTOR_DB[0][1]))}")

The vectors in this list of tuples are <class 'list'>


In [7]:
# Save the vector database to a txt file
if not vectorized_dataset_loaded:
  print('Saving the vector database to vector_db.txt')
  with open('vector_db.txt', 'w') as file:
    for chunk, embedding in VECTOR_DB:
      file.write(f"{chunk.strip()}\t{','.join(map(str, embedding))}\n")


## Information Retrival
Now we want to implement a function to retrieve information that is closest to the query for the LLM.<br>
First we need to make a function that calculates cosine similarity. <br>
Lets do it in a way that is fast (even though for this demo, we really don't need to)

In [34]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

def cosine_sim_sklearn_vecs(a, b):
    """
    Calculate cosine similarity between two vectors using sklearn.
    """
    a = np.array(a).reshape(1, -1)
    b = np.array(b).reshape(1, -1)
    return cosine_similarity(a, b)[0, 0]


In [37]:
v1 = [1, 2, 3]
v2 = [4, 5, 6]
print(cosine_similarity(v1, v2))  # should be ~0.9746

RecursionError: maximum recursion depth exceeded while calling a Python object

Now lets implement retrival

In [19]:
def retrieve(query, top_n=3):
    # Calculate the embedding for the query
    query_emb = ollama.embed(model=EMBEDDING_MODEL, input=query)['embeddings'][0]
    # Calculate cosine similarity for each chunk in the vector database
    similarities = [(chunk, cosine_similarity(query_emb, emb)) for chunk, emb in VECTOR_DB]
    # Sort the similarities in descending order
    similarities.sort(key=lambda x: x[1], reverse=True)
    return similarities[:top_n]


In [32]:
input_query = input('Ask me a question: ')
retrieved_knowledge = retrieve(input_query)

print('Retrieved knowledge:')
for chunk, similarity in retrieved_knowledge:
    print(f' - (similarity: {similarity:.2f}) {chunk}')

instruction_prompt = (
    "You are a helpful chatbot.\n"
    "Use only the following pieces of context to answer the question. "
    "Don't make up any new information:\n"
    + "\n".join([f' - {chunk}' for chunk, _ in retrieved_knowledge])
)

ValueError: Expected 2D array, got 1D array instead:
array=[-8.73156100e-02 -7.91125900e-02 -4.01601400e-02 -1.92160500e-02
  5.80570440e-02  2.30034240e-02  2.58869200e-02  6.24126230e-02
 -2.33088680e-02 -5.54876630e-02 -7.04705800e-02 -2.62286530e-02
 -5.67721870e-02  5.56011870e-03  6.52508400e-02  2.27959570e-02
  2.38364840e-02  3.74786200e-02 -5.18243500e-03 -4.59786540e-02
  1.25232320e-02 -4.97066160e-02 -7.94574900e-02  2.53541490e-03
  1.02479760e-02 -4.02277340e-04  2.54537350e-02 -1.42930640e-02
 -6.29645070e-03 -1.90851730e-02  8.49206760e-04 -1.66690440e-02
  3.20937450e-02 -1.29032400e-02 -2.26669100e-02 -8.04212700e-03
 -1.75521670e-02 -4.40231430e-03  1.29331050e-02 -2.82213070e-02
 -6.83303500e-02  1.22221910e-02  3.89088500e-02  4.91082040e-02
 -6.56129300e-02  2.29920100e-02 -2.19522320e-02 -1.19694950e-02
 -8.97855800e-03  9.62287300e-02 -1.38726330e-02  3.93324230e-02
 -5.11332100e-02 -2.68256420e-02  1.78182600e-02  4.88897600e-02
  5.36908170e-02 -1.54121870e-0